**Run the following two cells before you begin.**

In [2]:
%autosave 10

Autosaving every 10 seconds


In [1]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [3]:
# Import the data set
data=pd.read_csv("cleaned_data.csv")

In [8]:
def sigmoid(X):
    Y = 1 / (1 + np.exp(-X))
    return Y

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [7]:
# Create a train/test split
features=['PAY_1','LIMIT_BAL']
values=['default payment next month']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data[features].values,data[values].values,test_size=0.2,random_state=24)


______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [9]:
from sklearn.linear_model import LogisticRegression
example_lr = LogisticRegression(solver='liblinear')
example_lr

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [11]:
# Fit the logistic regression model on training data
example_lr.fit(X_train, y_train)

C:\Users\JETKING\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
# Make predictions using `.predict()`
y_pred = example_lr.predict(X_test)

In [15]:
# Find class probabilities using `.predict_proba()`
y_pred_proba=example_lr.predict_proba(X_test)

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [33]:
# Add column of 1s to features
np.ones((X_test.shape[0],1)).shape
ones_feature=np.hstack([np.ones((X_test.shape[0],1)),X_test])
ones_feature


array([[1.0e+00, 0.0e+00, 9.0e+04],
       [1.0e+00, 0.0e+00, 2.0e+04],
       [1.0e+00, 0.0e+00, 6.0e+04],
       ...,
       [1.0e+00, 0.0e+00, 4.0e+04],
       [1.0e+00, 0.0e+00, 4.1e+05],
       [1.0e+00, 0.0e+00, 5.0e+04]])

In [22]:
intercept_and_coefs=np.concatenate([example_lr.intercept_.reshape(1,1),example_lr.coef_],axis=1)
intercept_and_coefs

array([[-6.94678148e-11,  8.05451221e-11, -6.78127864e-06]])

In [24]:
# Get coefficients and intercepts from trained model
X_lin=np.dot(intercept_and_coefs,np.transpose(ones_feature))

In [25]:
# Manually calculate predicted probabilities
y_pred_proba=sigmoid(X_lin)

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [27]:
# Manually calculate predicted classes
y_pred_mannual=y_pred_proba>=0.5

In [28]:
# Compare to scikit-learn's predicted classes
y_pred.shape

(5396,)

In [29]:
y_pred_mannual.shape

(1, 5396)

In [30]:
np.array_equal(y_pred.reshape(1,-1),y_pred_mannual)

True

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [40]:
# Use scikit-learn's predicted probabilities to calculate ROC AU
import numpy as np
import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score
metrics.roc_auc_score(y_test, y_pred)

0.5

In [47]:
# Use manually calculated predicted probabilities to calculate ROC AUC
metrics.roc_auc_score(y_test,y_pred_mannual.reshape(y_pred_mannual.shape[1],))

0.5

In [50]:
y_pred_mannual.reshape(y_pred_mannual.shape[1],).shape

(5396,)